In [1]:
%%capture
!pip install datasets
!pip install git+https://github.com/huggingface/transformers.git
!pip install flax
!pip install git+https://github.com/deepmind/optax.git

In [2]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [3]:
jax.local_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [4]:
GLUE_TASKS=['cola','mnli','mnli-mm','mrpc','qnli','qqp',
            'rte','sst2','stsb','wnli']

In [5]:
task='cola'
model_checkpoint='bert-base-cased'
per_device_batch_size=4

In [6]:
from datasets import load_dataset,load_metric

In [7]:
raw_dataset=load_dataset('glue','cola')
metric=load_metric('glue','cola')

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [8]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [10]:
raw_dataset['train'][0]

{'idx': 0,
 'label': 1,
 'sentence': "Our friends won't buy this analysis, let alone the next one we propose."}

In [11]:
sentence1_key,sentence2_key=task_to_keys[task]

def preprocess_function(examples):
  texts=(
      (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key],examples[sentence2_key]))
  processed=tokenizer(*texts,padding='max_length',max_length=128,
                      truncation=True)
  processed['labels']=examples['label']
  return processed

In [12]:
tokenized_dataset=raw_dataset.map(preprocess_function,batched=True,remove_columns=raw_dataset['train'].column_names)

In [13]:
train_dataset=tokenized_dataset['train']
eval_dataset=tokenized_dataset['validation']

In [14]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [15]:
print(next(iter(train_dataset)))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3458, 2053, 1281, 112, 189, 4417, 1142, 3622, 117, 1519, 2041, 1103, 1397, 1141, 1195, 17794, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 1, 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [16]:
from transformers import FlaxAutoModelForSequenceClassification,AutoConfig

num_labels=2
seed=0
config=AutoConfig.from_pretrained(model_checkpoint,num_labels=num_labels)
model=FlaxAutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config,seed=seed)

Some weights of the model checkpoint at bert-base-cased were not used when initializing FlaxBertForSequenceClassification: {('cls', 'predictions', 'transform', 'dense', 'kernel'), ('cls', 'predictions', 'bias'), ('cls', 'predictions', 'transform', 'dense', 'bias'), ('cls', 'predictions', 'transform', 'LayerNorm', 'scale'), ('cls', 'predictions', 'transform', 'LayerNorm', 'bias')}
- This IS expected if you are initializing FlaxBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of FlaxBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased 

In [17]:
import jax
import flax
import optax

from itertools import chain
from tqdm.notebook import tqdm
from typing import Callable

import jax.numpy as jnp

from flax.training.common_utils import get_metrics,onehot,shard,shard_prng_key
from flax.training import train_state

In [18]:
num_train_epochs=3
learning_rate=2e-5

In [19]:
total_batch_size=per_device_batch_size*jax.local_device_count()
print(total_batch_size)

32


In [20]:
num_train_steps=len(train_dataset)//total_batch_size*num_train_epochs
print(num_train_steps)

801


In [21]:
learning_rate_function=optax.linear_schedule(init_value=learning_rate,end_value=0,transition_steps=num_train_steps)

In [22]:
print(learning_rate_function(count=1))

1.997503e-05


#Defining the training state

Next, we will create the training state that includes the optimizer, the loss function, and is responsible for updating the model's parameters during training.

Most JAX transformations (notably jax.jit) require functions that are transformed to have no side-effects. This is because any such side-effects will only be executed once, when the Python version of the function is run during compilation (see Stateful Computations in JAX). As a consequence, Flax models (which can be transformed by JAX transformations) are immutable, and the state of the model (i.e., its weight parameters) are stored outside of the model instance.

Models are initialized and updated in a purely functional way: you pass the state to the model when calling it, and the model returns the new (possibly modified) state, leaving the model instance itself unchanged.

Flax provides a convenience class flax.training.train_state.TrainState, which stores things such as the model parameters, the loss function, the optimizer, and exposes an apply_gradients function to update the model's weight parameters.

Alright, let's begin by defining our training state class. We create a derived TrainState class that additionally stores the model's forward pass as eval_function as well as a loss_function.

In [23]:
class TrainState(train_state.TrainState):
  logits_function:Callable=flax.struct.field(pytree_node=False)
  loss_function:Callable=flax.struct.field(pytree_node=False)

In [24]:
def adamw(weight_decay):
  return optax.adamw(learning_rate=learning_rate_function,b1=0.9,b2=0.99,eps=1e-6,weight_decay=weight_decay)

In [25]:
decay_path=lambda p:not any(x in p for x in ['bias','LayerNorm.weight'])
print(decay_path('bias'))

False


In [26]:
decay_path

<function __main__.<lambda>>

In [27]:
print(decay_path('a'))

True


In [28]:
xs={'foo':1,'bar':{'a': 2,'b':{}}}
flat_xs=flax.traverse_util.flatten_dict(xs)
print(flat_xs)

{('foo',): 1, ('bar', 'a'): 2}


In [29]:
def traverse(function):
  def mask(data):
    flat=flax.traverse_util.flatten_dict(data)
    return flax.traverse_util.unflatten_dict({k:function(k,v) for k,v in flat.items()})
  return mask

gradient_transformation=optax.chain(
    optax.masked(adamw(0.0),mask=traverse(lambda path,_:decay_path(path))),
    optax.masked(adamw(0.01),mask=traverse(lambda path,_:not decay_path(path)))
)

In [30]:
def add_one(x):
  return x+1

add_one(2)

3

In [31]:
add_one=lambda x: x + 1
add_one(2)

3

In [32]:
def loss_function(logits,labels):
  logits=flax.linen.log_softmax(logits)
  xentropy=optax.softmax_cross_entropy(logits,onehot(labels,num_classes=2))
  return jnp.mean(xentropy)

def eval_function(logits):
  return logits.argmax(-1)

In [33]:
state=TrainState.create(apply_fn=model.__call__,
                        params=model.params,
                        tx=gradient_transformation,
                        logits_function=eval_function,
                        loss_function=loss_function)

In [34]:
print(type(state))

<class '__main__.TrainState'>


#Defining the training and evaluation step

During fine-tuning, we want to update the model parameters and evaluate the performance after each epoch.

Let's write the functions train_step and eval_step accordingly. During training the weight parameters should be updated as follows:

1. Define a loss function loss_function that first runs a forward pass of the model given data input. Remember that Flax models are immutable, and we explicitly pass it the state (in this case the model parameters and the RNG). loss_function returns a scalar loss (using the previously defined state.loss_function) between the model output and input targets.

2. Differentiate this loss function using jax.value_and_grad. This is a JAX transformation called automatic differentiation, which computes the gradient of loss_function given the input to the function (i.e., the parameters of the model), and returns the value and the gradient in a pair (loss, gradients).

3. Compute the mean gradient over all devices using the collective operation lax.pmean. As we will see below, each device runs train_step on a different batch of data, but by taking the mean here we ensure the model parameters are the same on all devices.

4. Use state.apply_gradients, which applies the gradients to the weights.

In [35]:
def train_step(state,batch,dropout_rng):
  targets=batch.pop('labels')
  dropout_rng,new_dropout_rng=jax.random.split(dropout_rng)

  def loss_function(params):
    logits=state.apply_fn(**batch,params=params,dropout_rng=dropout_rng,train=True)[0]
    loss=state.loss_function(logits,targets)
    return loss

  grad_function=jax.value_and_grad(loss_function)
  loss,grad=grad_function(state.params)
  grad=jax.lax.pmean(grad,'batch')
  new_state=state.apply_gradients(grads=grad)
  metrics=jax.lax.pmean({'loss':loss,'learning_rate':learning_rate_function(state.step)},axis_name='batch')  
  return new_state,metrics,new_dropout_rng


In [36]:
def train_step(state, batch, dropout_rng):
    targets = batch.pop("labels")
    dropout_rng, new_dropout_rng = jax.random.split(dropout_rng)

    def loss_function(params):
        logits = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True)[0]
        loss = state.loss_function(logits, targets)
        return loss

    grad_function = jax.value_and_grad(loss_function)
    loss, grad = grad_function(state.params)
    grad = jax.lax.pmean(grad, "batch")
    new_state = state.apply_gradients(grads=grad)
    metrics = jax.lax.pmean({"loss": loss, "learning_rate": learning_rate_function(state.step)}, axis_name="batch")
    return new_state, metrics, new_dropout_rng

In [37]:
parallel_train_step=jax.pmap(train_step,axis_name='batch',donate_argnums=(0,))

In [38]:
def eval_step(state,batch):
  logits=state.apply_fn(**batch,params=state.params,train=False)[0]
  return state.logits_function(logits)

In [39]:
parallel_eval_step=jax.pmap(eval_step,axis_name='batch')

In [40]:
def glue_train_data_loader(rng,dataset,batch_size):
  steps_per_epoch=len(dataset)//batch_size
  perms=jax.random.permutation(rng,len(dataset))
  perms=perms[:steps_per_epoch*batch_size]
  perms=perms.reshape((steps_per_epoch,batch_size))

  for perm in perms:
    batch=dataset[perm]
    batch={k:jnp.array(v) for k,v in batch.items()}
    batch=shard(batch)

    yield batch

In [41]:
def glue_eval_data_loader(dataset,batch_size):
  for i in range(len(dataset)//batch_size):
    batch=dataset[i*batch_size:(i+1)*batch_size]
    batch={k:jnp.array(v) for k,v in batch.items()}
    batch=shard(batch)

    yield batch

Next, we replicate/copy the weight parameters on each device, so that we can pass them to our pmapped functions.

In [42]:
state=flax.jax_utils.replicate(state)

/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:317: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "
/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:304: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  "jax.host_id has been renamed to jax.process_index. This alias "


In [43]:
rng=jax.random.PRNGKey(seed)
print(rng)

[0 0]


In [44]:
dropout_rngs=jax.random.split(rng,jax.local_device_count())
print(dropout_rngs)

[[2292653872 4174357202]
 [2646725894 3731661025]
 [2709833381  877817354]
 [3184928187 3657760615]
 [ 931858003 1395580008]
 [3159319543 3403909611]
 [1595590687  561027531]
 [ 207801998 2609351694]]


In [45]:
print(jax.random.split(rng))

[[4146024105  967050713]
 [2718843009 1272950319]]


In [ ]:
for i,epoch in enumerate(tqdm(range(1,num_train_epochs+1),desc=f'Epoch ...',position=0,leave=True)):
  rng,input_rng=jax.random.split(rng)

  #train
  with tqdm(total=len(train_dataset)//total_batch_size,desc='Training...',leave=False) as progress_bar_train:
    for batch in glue_train_data_loader(input_rng,train_dataset,total_batch_size):
      state,train_metrics,dropout_rngs=parallel_train_step(state,batch,dropout_rngs)
      progress_bar_train.update(1)

  #evaluate
  with tqdm(total=len(eval_dataset)//total_batch_size,desc='Evaluating...',leave=False) as progress_bar_eval:
    for batch in glue_eval_data_loader(eval_dataset,total_batch_size):
      labels=batch.pop('labels')
      predictions=parallel_eval_step(state,batch)
      metric.add_batch(predictions=chain(*predictions),references=chain(*labels))
      progress_bar_eval.update(1)

  eval_metric=metric.compute()
  loss=round(flax.jax_utils.unreplicate(train_metrics)['loss'].item(),3)
  eval_score=round(list(eval_metric.values())[0],3)
  metric_name=list(eval_metric.keys())[0]

  print(f'{i+1}/{num_train_epochs} | Train loss:{loss} | Eval {metric_name}:{eval_score}')   